In [1]:
import numpy as np
import pandas as pd

from lib.script_scraper import ScriptScraper
from lib.corpus_enhancer import CorpusEnhancer

In [2]:
raw_dir = 'raw/'
final_dir = 'final/'

### Parks and Recreation

In [3]:
pnr_series_url = 'https://subslikescript.com/series/Parks_and_Recreation-1266020'
pnr_series_id = 'parksandrec'
acts = 3
scenes = 5
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'

In [3]:
pnr_scraper = ScriptScraper(
    series_url = pnr_series_url,
    series_id = pnr_series_id,
    user_agent = user_agent,
    acts = acts,
    scenes = scenes
)

In [4]:
pnr_corpus = pnr_scraper.generate_corpus()

In [5]:
pnr_corpus

line
series_id   season_id episode_id episode_title         scene_id line_id                                         
parksandrec 1         1          Pilot                 1        1                                     Hello. Hi.
                                                                2        My name is Leslie Knope, and I work for
                                                                3           the Parks and Recreation Department.
                                                                4                 Can I ask you a few questions?
                                                                5                    Would you say that you are,
...                                                                                                          ...
            7         12         One Last Ride: Part 1 15       135                             You ready, babe?
                                                                136                                         Yes.
                                                                137                                   I'm ready.
                                                                138                        That is a series wrap
                                                                139                   on "Parks and Recreation."

[107954 rows x 1 columns]

In [6]:
pnr_tokens = pnr_scraper.generate_tokens()

In [7]:
pnr_tokens

token_str  \
series_id   season_id episode_id episode_title         scene_id line_id token_id                 
parksandrec 1         1          Pilot                 1        1       0               Hello.   
                                                                        1                  Hi.   
                                                                2       0                   My   
                                                                        1                 name   
                                                                        2                   is   
...                                                                                        ...   
            7         12         One Last Ride: Part 1 15       138     4                 wrap   
                                                                139     0                   on   
                                                                        1               "Parks   
                                                                        2                  and   
                                                                        3         Recreation."   

                                                                                    term_str  
series_id   season_id episode_id episode_title         scene_id line_id token_id              
parksandrec 1         1          Pilot                 1        1       0              hello  
                                                                        1                 hi  
                                                                2       0                 my  
                                                                        1               name  
                                                                        2                 is  
...                                                                                      ...  
            7         12         One Last Ride: Part 1 15       138     4               wrap  
                                                                139     0                 on  
                                                                        1              parks  
                                                                        2                and  
                                                                        3         recreation  

[440560 rows x 2 columns]

In [8]:
pnr_vocab = pnr_scraper.generate_vocab()

In [9]:
pnr_vocab

,n
term_str,
i,14963
you,14103
the,11459
to,10396
a,10002
...,...
fourteen,1
misuse,1
bribery,1


In [10]:
pnr_corpus.to_csv(raw_dir + 'PNR-LINES.csv')
pnr_tokens.to_csv(raw_dir + 'PNR-TOKENS.csv')
pnr_vocab.to_csv(raw_dir + 'PNR-VOCAB.csv')

### Brooklyn Nine-Nine

In [4]:
b99_series_url = 'https://subslikescript.com/series/Brooklyn_Nine-Nine-2467372'
b99_series_id = 'brooklyn99'

In [12]:
b99_scraper = ScriptScraper(
    series_url = b99_series_url,
    series_id = b99_series_id,
    user_agent = user_agent,
    acts = acts,
    scenes = scenes
)

In [13]:
b99_corpus = b99_scraper.generate_corpus()

In [14]:
b99_corpus

line
series_id  season_id episode_id episode_title     scene_id line_id                              
brooklyn99 1         1          Pilot             1        1        This job is eating me alive.
                                                           2            I can't breathe anymore.
                                                           3             I spent all these years
                                                           4          trying to be the good guy.
                                                           5                    Ripped By mstoll
...                                                                                          ...
           8         10         The Last Day Pt 2 15       89                           Captain?
                                                           90                 Are we doing this?
                                                           91                   What I say is...
                                                           92                         Nine-Nine!
                                                           93                         Nine-Nine!

[142930 rows x 1 columns]

In [15]:
b99_tokens = b99_scraper.generate_tokens()

In [16]:
b99_tokens

token_str  \
series_id  season_id episode_id episode_title     scene_id line_id token_id               
brooklyn99 1         1          Pilot             1        1       0               This   
                                                                   1                job   
                                                                   2                 is   
                                                                   3             eating   
                                                                   4                 me   
...                                                                                 ...   
           8         10         The Last Day Pt 2 15       91      1                  I   
                                                                   2                say   
                                                                   3              is...   
                                                           92      0         Nine-Nine!   
                                                           93      0         Nine-Nine!   

                                                                             term_str  
series_id  season_id episode_id episode_title     scene_id line_id token_id            
brooklyn99 1         1          Pilot             1        1       0             this  
                                                                   1              job  
                                                                   2               is  
                                                                   3           eating  
                                                                   4               me  
...                                                                               ...  
           8         10         The Last Day Pt 2 15       91      1                i  
                                                                   2              say  
                                                                   3               is  
                                                           92      0         ninenine  
                                                           93      0         ninenine  

[586247 rows x 2 columns]

In [17]:
b99_vocab = b99_scraper.generate_vocab()

In [18]:
b99_vocab

,n
term_str,
,19630
i,18524
you,18007
the,16589
a,12511
...,...
reorganization,1
blizzards,1
penisremoval,1


In [29]:
b99_corpus.to_csv(raw_dir + 'B99-LINES.csv')
b99_tokens.to_csv(raw_dir + 'B99-TOKENS.csv')
b99_vocab.to_csv(raw_dir + 'B99-VOCAB.csv')

### The Office

In [5]:
office_series_url = 'https://subslikescript.com/series/The_Office-386676'
office_series_id = 'theoffice'

In [21]:
office_scraper = ScriptScraper(
    series_url = office_series_url,
    series_id = office_series_id,
    user_agent = user_agent,
    acts = acts,
    scenes = scenes
)

In [22]:
office_corpus = office_scraper.generate_corpus()

In [23]:
office_corpus

line
series_id season_id episode_id episode_title scene_id line_id                                
theoffice 1         1          Pilot         1        1                    All right Jim your
                                                      2           quarterlies look very good.
                                                      3                      How the thing is
                                                      4                going at the library ?
                                                      5                         Oh I told you
...                                                                                       ...
          9         23         Finale        15       99                  was a great subject
                                                      100                  for a documentary.
                                                      101             There's a lot of beauty
                                                      102                 in ordinary things.
                                                      103      Isn't that kind of the point? 

[146125 rows x 1 columns]

In [24]:
office_tokens = office_scraper.generate_tokens()

In [25]:
office_tokens

token_str  \
series_id season_id episode_id episode_title scene_id line_id token_id                
theoffice 1         1          Pilot         1        1       0                 All   
                                                              1               right   
                                                              2                 Jim   
                                                              3                your   
                                                      2       0         quarterlies   
...                                                                             ...   
          9         23         Finale        15       103     1                that   
                                                              2                kind   
                                                              3                  of   
                                                              4                 the   
                                                              5              point?   

                                                                           term_str  
series_id season_id episode_id episode_title scene_id line_id token_id               
theoffice 1         1          Pilot         1        1       0                 all  
                                                              1               right  
                                                              2                 jim  
                                                              3                your  
                                                      2       0         quarterlies  
...                                                                             ...  
          9         23         Finale        15       103     1                that  
                                                              2                kind  
                                                              3                  of  
                                                              4                 the  
                                                              5               point  

[563670 rows x 2 columns]

In [26]:
office_vocab = office_scraper.generate_vocab()

In [27]:
office_vocab

,n
term_str,
i,19894
you,19788
the,14497
to,13821
a,12686
...,...
bearings,1
conversing,1
upperdecker,1


In [28]:
office_corpus.to_csv(raw_dir + 'OFFICE-LINES.csv')
office_tokens.to_csv(raw_dir + 'OFFICE-TOKENS.csv')
office_vocab.to_csv(raw_dir + 'OFFICE-VOCAB.csv')

# Create LIB table

Info for the LIB tables are extracted from the following links:

- [Parks and Recreation](https://en.wikipedia.org/wiki/Parks_and_Recreation)
- [The Office](https://en.wikipedia.org/wiki/The_Office_(American_TV_series)
- [Brooklyn Nine-Nine](https://en.wikipedia.org/wiki/Brooklyn_Nine-Nine)


In [8]:
lib_headers = ['season_id', 'num_episodes', 'year', 'viewers_millions', 'rt_rating']
lib_index = ['series_id', 'season_id']
pnr_lib_values = [
    [1, 6, 2009, 6.00, 68],
    [2, 24, 2010, 4.60, 96],
    [3, 16, 2011, 5.10, 100],
    [4, 22, 2012, 4.40, 100],
    [5, 22, 2013, 4.06, 96],
    [6, 22, 2014, 3.76, 96],
    [7, 13, 2015, 4.57, 89]
]
pnr_lib = pd.DataFrame(pnr_lib_values, columns = lib_headers)
pnr_lib['series_id'] = pnr_series_id
pnr_lib['series_name'] = 'Parks and Recreation'
pnr_lib = pnr_lib.set_index(lib_index)

office_lib_values = [
    [1, 6, 2005, 5.40, 69],
    [2, 22, 2006, 8.00, 100],
    [3, 25, 2007, 8.30, 100],
    [4, 19, 2008, 8.04, 83],
    [5, 28, 2009, 9.04, 100],
    [6, 26, 2010, 8.73, 73],
    [7, 26, 2011, 7.73, 83],
    [8, 24, 2012, 6.51, 44],
    [9, 25, 2013, 5.06, 79]
]
office_lib = pd.DataFrame(office_lib_values, columns = lib_headers)
office_lib['series_id'] = office_series_id
office_lib['series_name'] = 'The Office'
office_lib = office_lib.set_index(lib_index)

b99_lib_values = [
    [1, 22, 2014, 4.80, 89],
    [2, 23, 2015, 4.87, 100],
    [3, 23, 2016, 3.98, 93],
    [4, 22, 2017, 2.87, 100],
    [5, 22, 2018, 2.71, 100],
    [6, 18, 2019, 3.11, 100],
    [7, 13, 2020, 2.69, 89],
    [8, 10, 2021, 1.69, 89]
]
b99_lib = pd.DataFrame(b99_lib_values, columns = lib_headers)
b99_lib['series_id'] = b99_series_id
b99_lib['series_name'] = 'Brooklyn Nine-Nine'
b99_lib = b99_lib.set_index(lib_index)

In [9]:
pnr_lib

num_episodes  year  viewers_millions  rt_rating  \
series_id   season_id                                                    
parksandrec 1                     6  2009              6.00         68   
            2                    24  2010              4.60         96   
            3                    16  2011              5.10        100   
            4                    22  2012              4.40        100   
            5                    22  2013              4.06         96   
            6                    22  2014              3.76         96   
            7                    13  2015              4.57         89   

                                series_name  
series_id   season_id                        
parksandrec 1          Parks and Recreation  
            2          Parks and Recreation  
            3          Parks and Recreation  
            4          Parks and Recreation  
            5          Parks and Recreation  
            6          Parks and Recreation  
            7          Parks and Recreation

In [13]:
pnr_lib.to_csv(raw_dir + 'PNR-LIB.csv')

In [10]:
b99_lib

num_episodes  year  viewers_millions  rt_rating  \
series_id  season_id                                                    
brooklyn99 1                    22  2014              4.80         89   
           2                    23  2015              4.87        100   
           3                    23  2016              3.98         93   
           4                    22  2017              2.87        100   
           5                    22  2018              2.71        100   
           6                    18  2019              3.11        100   
           7                    13  2020              2.69         89   
           8                    10  2021              1.69         89   

                             series_name  
series_id  season_id                      
brooklyn99 1          Brooklyn Nine-Nine  
           2          Brooklyn Nine-Nine  
           3          Brooklyn Nine-Nine  
           4          Brooklyn Nine-Nine  
           5          Brooklyn Nine-Nine  
           6          Brooklyn Nine-Nine  
           7          Brooklyn Nine-Nine  
           8          Brooklyn Nine-Nine

In [14]:
b99_lib.to_csv(raw_dir + 'B99-LIB.csv')

In [11]:
office_lib

num_episodes  year  viewers_millions  rt_rating  \
series_id season_id                                                    
theoffice 1                     6  2005              5.40         69   
          2                    22  2006              8.00        100   
          3                    25  2007              8.30        100   
          4                    19  2008              8.04         83   
          5                    28  2009              9.04        100   
          6                    26  2010              8.73         73   
          7                    26  2011              7.73         83   
          8                    24  2012              6.51         44   
          9                    25  2013              5.06         79   

                    series_name  
series_id season_id              
theoffice 1          The Office  
          2          The Office  
          3          The Office  
          4          The Office  
          5          The Office  
          6          The Office  
          7          The Office  
          8          The Office  
          9          The Office

In [15]:
office_lib.to_csv(raw_dir + 'OFFICE-LIB.csv')

# Add Metrics to CORPUS & VOCAB

In [5]:
OHCO = ['series_id', 'season_id', 'episode_id', 'scene_id', 'line_id', 'token_id']
TOKENS = OHCO[:6]
LINES = OHCO[:5]
SCENES = OHCO[:4]
EPISODES = OHCO[:3]
SEASONS = OHCO[:2]
SERIES = OHCO[:1]

### Parks and Recreation

In [3]:
pnr_tokens_raw = pd.read_csv(raw_dir + 'PNR-TOKENS.csv')

In [6]:
pnr_tokens_raw = pnr_tokens_raw.set_index(TOKENS).drop('episode_title', axis = 1)

In [7]:
pnr_tokens_raw

token_str  \
series_id   season_id episode_id scene_id line_id token_id                 
parksandrec 1         1          1        1       0               Hello.   
                                                  1                  Hi.   
                                          2       0                   My   
                                                  1                 name   
                                                  2                   is   
...                                                                  ...   
            7         12         15       138     4                 wrap   
                                          139     0                   on   
                                                  1               "Parks   
                                                  2                  and   
                                                  3         Recreation."   

                                                              term_str  
series_id   season_id episode_id scene_id line_id token_id              
parksandrec 1         1          1        1       0              hello  
                                                  1                 hi  
                                          2       0                 my  
                                                  1               name  
                                                  2                 is  
...                                                                ...  
            7         12         15       138     4               wrap  
                                          139     0                 on  
                                                  1              parks  
                                                  2                and  
                                                  3         recreation  

[440560 rows x 2 columns]

In [8]:
pnr_corp_enhancer = CorpusEnhancer(corpus = pnr_tokens_raw, ohco = TOKENS)

In [9]:
pnr_corpus_final = pnr_corp_enhancer.enhance_corpus(
    strip_hyphens = True, 
    strip_whitespace = True, 
    use_nltk = True
)

In [10]:
pnr_corpus_final

pos_tuple  \
series_id   season_id episode_id scene_id line_id token_id                       
parksandrec 1         1          1        1       0               (Hello., NN)   
                                                  1                  (Hi., NN)   
                                          2       0                 (My, PRP$)   
                                                  1                 (name, NN)   
                                                  2                  (is, VBZ)   
...                                                                        ...   
            7         12         15       138     4                 (wrap, NN)   
                                          139     0                   (on, IN)   
                                                  1              ("Parks, NNS)   
                                                  2                  (and, CC)   
                                                  3         (Recreation.", NN)   

                                                             pos  \
series_id   season_id episode_id scene_id line_id token_id         
parksandrec 1         1          1        1       0           NN   
                                                  1           NN   
                                          2       0         PRP$   
                                                  1           NN   
                                                  2          VBZ   
...                                                          ...   
            7         12         15       138     4           NN   
                                          139     0           IN   
                                                  1          NNS   
                                                  2           CC   
                                                  3           NN   

                                                               token_str  \
series_id   season_id episode_id scene_id line_id token_id                 
parksandrec 1         1          1        1       0               Hello.   
                                                  1                  Hi.   
                                          2       0                   My   
                                                  1                 name   
                                                  2                   is   
...                                                                  ...   
            7         12         15       138     4                 wrap   
                                          139     0                   on   
                                                  1               "Parks   
                                                  2                  and   
                                                  3         Recreation."   

                                                              term_str  
series_id   season_id episode_id scene_id line_id token_id              
parksandrec 1         1          1        1       0              hello  
                                                  1                 hi  
                                          2       0                 my  
                                                  1               name  
                                                  2                 is  
...                                                                ...  
            7         12         15       138     4               wrap  
                                          139     0                 on  
                                                  1              parks  
                                                  2                and  
                                                  3         recreation  

[436872 rows x 4 columns]

In [16]:
pnr_corpus_final.to_csv(final_dir + 'PNR-CORPUS.csv')

#### Compute BOW using SCENES

In [13]:
pnr_vocab_final = pnr_corp_enhancer.generate_enhanced_vocab(ohco = SCENES)

Using TF method: sum


In [14]:
pnr_vocab_final

,n,n_chars,p,i,h,max_pos,n_pos,cat_pos,stop,tfidf_mean,tfidf_sum,tfidf_median,tfidf_max,dfidf
term_str,,,,,,,,,,,,,,
0,4,1,0.000009,16.736752,0.000153,CD,2,"{NN, CD}",0,0.037646,0.150586,0.038765,0.039454,35.350512
000003,1,6,0.000002,18.736752,0.000043,NN,1,{NN},0,0.044055,0.044055,0.044055,0.044055,10.837628
05,1,2,0.000002,18.736752,0.000043,NNS,1,{NNS},0,0.042836,0.042836,0.042836,0.042836,10.837628
099twposter,1,11,0.000002,18.736752,0.000043,NN,1,{NN},0,0.045922,0.045922,0.045922,0.045922,10.837628
1,28,1,0.000064,13.929397,0.000893,CD,1,{CD},0,0.039243,0.824097,0.029164,0.073661,135.351521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zut,1,3,0.000002,18.736752,0.000043,NN,1,{NN},0,0.051608,0.051608,0.051608,0.051608,10.837628
zuzu,2,4,0.000005,17.736752,0.000081,NN,1,{NN},0,0.052207,0.104414,0.052207,0.053176,19.675256
zuzus,1,5,0.000002,18.736752,0.000043,NN,1,{NN},0,0.056446,0.056446,0.056446,0.056446,10.837628


In [17]:
pnr_vocab_final.to_csv(final_dir + 'PNR-VOCAB-SCENES.csv')

### Brooklyn Nine-Nine

In [18]:
b99_tokens_raw = pd.read_csv(raw_dir + 'B99-TOKENS.csv')
b99_tokens_raw = b99_tokens_raw.set_index(TOKENS).drop('episode_title', axis = 1)
b99_tokens_raw

token_str  \
series_id  season_id episode_id scene_id line_id token_id               
brooklyn99 1         1          1        1       0               This   
                                                 1                job   
                                                 2                 is   
                                                 3             eating   
                                                 4                 me   
...                                                               ...   
           8         10         15       91      1                  I   
                                                 2                say   
                                                 3              is...   
                                         92      0         Nine-Nine!   
                                         93      0         Nine-Nine!   

                                                           term_str  
series_id  season_id episode_id scene_id line_id token_id            
brooklyn99 1         1          1        1       0             this  
                                                 1              job  
                                                 2               is  
                                                 3           eating  
                                                 4               me  
...                                                             ...  
           8         10         15       91      1                i  
                                                 2              say  
                                                 3               is  
                                         92      0         ninenine  
                                         93      0         ninenine  

[586247 rows x 2 columns]

In [19]:
b99_corp_enhancer = CorpusEnhancer(corpus = b99_tokens_raw, ohco = TOKENS)
b99_corpus_final = b99_corp_enhancer.enhance_corpus(
    strip_hyphens = True, 
    strip_whitespace = True, 
    use_nltk = True
)
b99_corpus_final

pos_tuple  pos  \
series_id  season_id episode_id scene_id line_id token_id                       
brooklyn99 1         1          1        1       0            (This, DT)   DT   
                                                 1             (job, NN)   NN   
                                                 2             (is, VBZ)  VBZ   
                                                 3         (eating, VBG)  VBG   
                                                 4             (me, PRP)  PRP   
...                                                                  ...  ...   
           8         10         15       91      3           (is..., NN)   NN   
                                         92      0            (Nine, CD)   CD   
                                                 0          (Nine!, NNP)  NNP   
                                         93      0            (Nine, CD)   CD   
                                                 0          (Nine!, NNP)  NNP   

                                                          token_str term_str  
series_id  season_id episode_id scene_id line_id token_id                     
brooklyn99 1         1          1        1       0             This     this  
                                                 1              job      job  
                                                 2               is       is  
                                                 3           eating   eating  
                                                 4               me       me  
...                                                             ...      ...  
           8         10         15       91      3            is...       is  
                                         92      0             Nine     nine  
                                                 0            Nine!     nine  
                                         93      0             Nine     nine  
                                                 0            Nine!     nine  

[570367 rows x 4 columns]

In [20]:
b99_corpus_final.to_csv(final_dir + 'B99-CORPUS.csv')

#### Compute BOW using SCENES

In [21]:
b99_vocab_final = b99_corp_enhancer.generate_enhanced_vocab(ohco = SCENES)
b99_vocab_final

Using TF method: sum


,n,n_chars,p,i,h,max_pos,n_pos,cat_pos,stop,tfidf_mean,tfidf_sum,tfidf_median,tfidf_max,dfidf
term_str,,,,,,,,,,,,,,
0,6,1,0.000011,16.536447,0.000174,CD,1,{CD},0,0.030262,0.181570,0.029397,0.036890,51.129603
00,1,2,0.000002,19.121410,0.000034,CD,1,{CD},0,0.042070,0.042070,0.042070,0.042070,11.106563
000007044,1,9,0.000002,19.121410,0.000034,CD,1,{CD},0,0.041754,0.041754,0.041754,0.041754,11.106563
001834310,1,9,0.000002,19.121410,0.000034,CD,1,{CD},0,0.037146,0.037146,0.037146,0.037146,11.106563
01,2,2,0.000004,18.121410,0.000064,CD,1,{CD},0,0.037614,0.075227,0.037614,0.041084,20.213126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zwee,2,4,0.000004,18.121410,0.000064,NN,2,"{NN, NNP}",0,0.091412,0.091412,0.091412,0.091412,11.106563
à,6,1,0.000011,16.536447,0.000174,NN,2,"{NN, CD}",0,0.032450,0.194698,0.031625,0.039270,51.129603
âª,5,2,0.000009,16.799481,0.000147,NN,1,{NN},0,0.050788,0.203151,0.039702,0.086729,36.426252


In [22]:
b99_vocab_final.to_csv(final_dir + 'B99-VOCAB-SCENES.csv')

### The Office

In [23]:
office_tokens_raw = pd.read_csv(raw_dir + 'OFFICE-TOKENS.csv')
office_tokens_raw = office_tokens_raw.set_index(TOKENS).drop('episode_title', axis = 1)
office_tokens_raw

token_str  \
series_id season_id episode_id scene_id line_id token_id                
theoffice 1         1          1        1       0                 All   
                                                1               right   
                                                2                 Jim   
                                                3                your   
                                        2       0         quarterlies   
...                                                               ...   
          9         23         15       103     1                that   
                                                2                kind   
                                                3                  of   
                                                4                 the   
                                                5              point?   

                                                             term_str  
series_id season_id episode_id scene_id line_id token_id               
theoffice 1         1          1        1       0                 all  
                                                1               right  
                                                2                 jim  
                                                3                your  
                                        2       0         quarterlies  
...                                                               ...  
          9         23         15       103     1                that  
                                                2                kind  
                                                3                  of  
                                                4                 the  
                                                5               point  

[563670 rows x 2 columns]

In [24]:
office_corp_enhancer = CorpusEnhancer(corpus = office_tokens_raw, ohco = TOKENS)
office_corpus_final = office_corp_enhancer.enhance_corpus(
    strip_hyphens = True, 
    strip_whitespace = True, 
    use_nltk = True
)
office_corpus_final

pos_tuple  \
series_id season_id episode_id scene_id line_id token_id                       
theoffice 1         1          1        1       0                  (All, DT)   
                                                1                (right, NN)   
                                                2                 (Jim, NNP)   
                                                3               (your, PRP$)   
                                        2       0         (quarterlies, NNS)   
...                                                                      ...   
          9         23         15       103     1                 (that, IN)   
                                                2                 (kind, NN)   
                                                3                   (of, IN)   
                                                4                  (the, DT)   
                                                5               (point?, NN)   

                                                           pos    token_str  \
series_id season_id episode_id scene_id line_id token_id                      
theoffice 1         1          1        1       0           DT          All   
                                                1           NN        right   
                                                2          NNP          Jim   
                                                3         PRP$         your   
                                        2       0          NNS  quarterlies   
...                                                        ...          ...   
          9         23         15       103     1           IN         that   
                                                2           NN         kind   
                                                3           IN           of   
                                                4           DT          the   
                                                5           NN       point?   

                                                             term_str  
series_id season_id episode_id scene_id line_id token_id               
theoffice 1         1          1        1       0                 all  
                                                1               right  
                                                2                 jim  
                                                3                your  
                                        2       0         quarterlies  
...                                                               ...  
          9         23         15       103     1                that  
                                                2                kind  
                                                3                  of  
                                                4                 the  
                                                5               point  

[564628 rows x 4 columns]

In [25]:
office_corpus_final.to_csv(final_dir + 'OFFICE-CORPUS.csv')

#### Compute BOW using SCENES

In [26]:
office_vocab_final = office_corp_enhancer.generate_enhanced_vocab(ohco = SCENES)
office_vocab_final

Using TF method: sum


,n,n_chars,p,i,h,max_pos,n_pos,cat_pos,stop,tfidf_mean,tfidf_sum,tfidf_median,tfidf_max,dfidf
term_str,,,,,,,,,,,,,,
0,7,1,0.000012,16.297894,0.000202,CD,1,{CD},0,0.061431,0.307155,0.046196,0.110984,45.503312
00983,1,5,0.000002,19.105249,0.000034,CD,1,{CD},0,0.060758,0.060758,0.060758,0.060758,11.422590
019,3,3,0.000005,17.520286,0.000093,CD,1,{CD},0,0.081009,0.162018,0.081009,0.108569,20.845181
02,3,2,0.000005,17.520286,0.000093,CD,1,{CD},0,0.182275,0.182275,0.182275,0.182275,11.422590
05,3,2,0.000005,17.520286,0.000093,CD,1,{CD},0,0.196941,0.196941,0.196941,0.196941,11.422590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
üdwight,1,7,0.000002,19.105249,0.000034,NN,1,{NN},0,0.053377,0.053377,0.053377,0.053377,11.422590
ühumming,1,8,0.000002,19.105249,0.000034,NN,1,{NN},0,0.059184,0.059184,0.059184,0.059184,11.422590
ümusical,4,8,0.000007,17.105249,0.000121,NN,1,{NN},0,0.113948,0.227896,0.113948,0.172750,20.845181


In [27]:
office_vocab_final.to_csv(final_dir + 'OFFICE-VOCAB-SCENES.csv')